# PyTorch

[PyTorch](https://en.wikipedia.org/wiki/PyTorch) is a machine learning framework based on the Torch library, used for applications such as computer vision and natural language processing. PyTorch provides two high-level features:

- Tensor computing with acceleration via graphics processing units (GPUs)
- Deep neural networks built on a tape-based automatic differentiation system

In [19]:
import torch

## Tensors

The central data abstraction in PyTorch is given by the `torch.tensor` class. It represents the counterpart of the `numpy.ndarray` class in NumPy, and many of the respective class methods have similar syntax.

### Tensor creation

Ways to create PyTorch tensors include:

- `torch.tensor()`
- `torch.empty()`
- `torch.zeros()`
- `torch.ones()`
- `torch.rand()`

In [15]:
a = torch.rand(3, 3, dtype=torch.float32)

In [16]:
print(a)

tensor([[0.2181, 0.5665, 0.1810],
        [0.2315, 0.9513, 0.4370],
        [0.2049, 0.1661, 0.2148]])


By default, PyTorch tensors are populated with 32-bit (single precision) floating point numbers suitable for arithmetic operations on GPUs, but many other data types are available and include:

- `torch.bool`
- `torch.int8`
- `torch.int16`
- `torch.int32`
- `torch.int64`
- `torch.half` or `torch.float16`
- `torch.float`
- `torch.double` or `torch.float64`

A PyTorch tensor can be converted to a regular Python list.

In [4]:
a.tolist()

[[0.6990807056427002, 0.6508345603942871, 0.6472038626670837],
 [0.30571407079696655, 0.07050120830535889, 0.4252395033836365],
 [0.6484412550926208, 0.5886563062667847, 0.9803081154823303]]

Conversely, a Python list can be converted to a PyTorch tensor.

In [5]:
torch.tensor(a.tolist())

tensor([[0.6991, 0.6508, 0.6472],
        [0.3057, 0.0705, 0.4252],
        [0.6484, 0.5887, 0.9803]])

### Tensor operations

PyTorch tensors have over three hundred operations that can be performed on them, including:

- `torch.abs()`
- `torch.max()`
- `torch.mean()`
- `torch.std()`
- `torch.prod()`
- `torch.unique()`
- `torch.matmul()`
- `torch.svd()`
- `torch.sin()`
- `torch.cos()`
- `torch.flatten()`

In [6]:
a.mean()

tensor(0.5573)

Note that a tensor with a scalar number is given in return. To instead get a Python number in return, we can perform

In [7]:
a.mean().item()

0.5573310852050781

### NumPy bridge

In [20]:
import numpy as np

In [21]:
np_array = np.ones((2, 3))
pth_tensor = torch.from_numpy(np_array)

In [10]:
print(pth_tensor)

tensor([[1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)


We note that the NumPy array default data type of float64 (double precision) is preserved. In fact, we merely created a pointer to the same data in memory such that a change in one object is reflected in both.

In [12]:
np_array[1, 2] = 2

In [14]:
print("Modified numpy array:\n", np_array)
print("Bridged pytorch tensor:\n", pth_tensor)

Modified numpy array:
 [[1. 1. 1.]
 [1. 1. 2.]]
Bridged pytorch tensor:
 tensor([[1., 1., 1.],
        [1., 1., 2.]], dtype=torch.float64)


A reason to create a bridge between data can e.g. be to take advantage of the easy accessible  GPU acceleration available in PyTorch for scientific codes developed with NumPy. 

## Neural networks

The machine learning models in PyTorch are built as neural networks with layers of neurons. The input layers received input data; hidden layers transforms the data; and the output layer provides the results upon which model predictions are made.

Every neuron has an associated activation level. The input level apart, activation levels in a given level, say $L$, are determined from those in the previous layer by use of weights that are collected in a matrix $\boldsymbol{W}$ and biases that are collected in a row vector $\boldsymbol{b}$. 

A layer is referred to as *linear* if the weights and biases are applied in a linear transformation

$$
\boldsymbol{a}^{(L)} = f(\boldsymbol{a}^{(L-1)} \boldsymbol{W}^T 
+ \boldsymbol{b})
$$

As indicated, to get the final activation levels also involves the elementwise operation of a  (typically) nonlinear activation function, $f$.

![Neural Network](../images/Colored_neural_network.svg)

### Flatten tensors

PyTorch receives input data in the form of batches of PyTorch tensors of rank 1. If data is stored as tensors of higher rank, they first need to be flattened. 

Let us assume having three $2\times 2$ tensors as input, e.g. three greyscale images of two-by-two pixels.

In [117]:
batch_size = 3

tensor_batch = torch.rand(batch_size, 2, 2)

In [118]:
flatten = torch.nn.Flatten()

In [133]:
a0 = flatten(tensor_batch)

These have now been flattened to become three row vectors of dimension four.

In [134]:
print(a0)

tensor([[0.7951, 0.5572, 0.1773, 0.1704],
        [0.2147, 0.7379, 0.1298, 0.5695],
        [0.7706, 0.5067, 0.2309, 0.8878]])


### Layer transformations

The linear layer transformation described above is achieved with the `torch.nn.Linear` class.

In [121]:
linear_layer = torch.nn.Linear(4, 2, bias=True)

When instantiated, the layer object receives weight and bias attributed that are initialized randomly. Here we consider a transformation from an input layer with four neurons to a hidden layer with only two.

In [122]:
linear_layer.weight

Parameter containing:
tensor([[ 0.4356,  0.0209, -0.4890, -0.3637],
        [-0.2066, -0.1390,  0.0430,  0.2331]], requires_grad=True)

In [123]:
linear_layer.bias

Parameter containing:
tensor([0.0178, 0.1455], requires_grad=True)

Use PyToch to perform the layer transoformation.

In [141]:
intermediate_result = linear_layer(a0)

In [142]:
intermediate_result

tensor([[ 0.2271, -0.0488],
        [-0.1438,  0.1369],
        [-0.0717,  0.1328]], grad_fn=<AddmmBackward0>)

Check the transformation with an explicit calculation of the linear transformation:

$$
\boldsymbol{a}^{(0)} \boldsymbol{W}^T 
+ \boldsymbol{b}
$$

In [143]:
torch.matmul(a0, linear_layer.weight.T) + linear_layer.bias

tensor([[ 0.2271, -0.0488],
        [-0.1438,  0.1369],
        [-0.0717,  0.1328]], grad_fn=<AddBackward0>)

We note that the two results are identical.

Now remains the application of the nonlinear activation function, $f$, according to

$$
\boldsymbol{a}^{(1)} =f( 
\boldsymbol{a}^{(0)} \boldsymbol{W}^T 
+ \boldsymbol{b})
$$

A common choice in machine learning is to adopt the rectifier linear unit function

$$
\mathrm{ReLU}(x) = \max(0,x) = \frac{x + |x|}{2}
$$

In [144]:
relu = torch.nn.ReLU()

In [145]:
a1 = relu(intermediate_result)

In [146]:
a1

tensor([[0.2271, 0.0000],
        [0.0000, 0.1369],
        [0.0000, 0.1328]], grad_fn=<ReluBackward0>)

The effect of the ReLU function is as anticipated.

## Loss function

## Backward propagation